In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from src.main import complete_builder
from src.utils import get_bearer_token

bearer_token = get_bearer_token()

In [3]:
complete_builder(
    creature_description="A robotic cowboy",
    creature_name="T3X",
    creature_type="Nemesis",
    setting_description="A standard D&D setting",
    setting_specific_skills=[],
    setting_removed_skils=[],
    skills=["Ranged (Heavy)", "Perception", "Cool"],
    allow_other_skills=True,
    combat_cr=12,
    social_cr=4,
    general_cr=7,
    brawn=-1,
    agility=-1,
    intellect=-1,
    cunning=-1,
    willpower=-1,
    presence=-1,
    number_of_weapons=3,
    number_of_abilities=3,
    bearer_token=bearer_token,
)

Building creature template
Filling character template with OpenAI, expect this to take about 15 seconds.
Validating character data
passes all creature validation steps
Building weapon template
Filling weapon template with OpenAI, expect this to take about 15 seconds.
Validating weapon data
passes all weapon validation steps
passes all weapon validation steps
passes all weapon validation steps
Adding weapons to character
Building abilities template
Filling ability template with OpenAI, expect this to take about 15 seconds.
Validating ability data
this passes all talent/ability validation steps
this passes all talent/ability validation steps
this passes all talent/ability validation steps
Adding abilities to character
Writing character to file
Uploading character to RPGSessions
[{'name': 'T3X', 'id': '64988e4418fddf5357491b6e'}]
Getting character from RPGSessions to edit raw data
Editing character data
Re-uploading character data to RPGSessions, overwriting existing character we just mad

{'customArchetypes': [{'name': 'NPC Archetype',
   'Brawn': 1,
   'Agility': 1,
   'Intellect': 1,
   'Cunning': 1,
   'Willpower': 1,
   'Presence': 1,
   'woundThreshold': 5,
   'strainThreshold': 1,
   'experience': '1',
   'description': '',
   'setting': ['All'],
   'skills': {},
   'talents': [],
   'id': 'ByO058vfgGB33EOrabeE'}],
 'customCareers': [{'name': 'NPC Career',
   'description': '',
   'setting': ['All'],
   'skills': [],
   'id': '9VWOCKS9chsyHcBjS1U3'}],
 'characters': [{'name': 'T3X',
   'careerSkillsRank': ['RangedHeavy',
    'Perception',
    'Cool',
    'Mechanics',
    'Resilience',
    'KnowledgeOuterRim'],
   'masterSkills': {'RangedHeavy': {'careerRank': 4},
    'Perception': {'careerRank': 3},
    'Cool': {'careerRank': 3},
    'Mechanics': {'careerRank': 2},
    'Resilience': {'careerRank': 2},
    'KnowledgeOuterRim': {'careerRank': 1}},
   'creatureType': 'Nemesis',
   'adversaryLevel': 3,
   'minionCount': 0,
   'creationCharacteristics': {'Brawn': 2,
  

In [4]:
from src.utils import get_character
c = get_character("64988c5b18fddf535748fcf3", bearer_token=bearer_token)

In [5]:
c

{'_id': '64988c5b18fddf535748fcf3',
 'createdDate': '2023-06-25T18:50:03.878Z',
 'modifiedDate': '2023-06-25T18:50:04.378Z',
 'status': 1,
 'name': 'Buzz Lightyear',
 'lookupName': 'buzz lightyear',
 'image': 'https://unbound-legends.imgix.net/character2/genesys-T-jjr.jpg?auto=format',
 'notes': '',
 'privateNotes': '',
 'rulesSystem': '[rules] nds',
 'characterType': '[character type] npc',
 'configuration': {'diceTheme': '[dice theme] genesys',
  'gameTheme': '[game theme] genesys terrinoth',
  'ndsCharacterType': '[nds character type] nemesis',
  'talentsDisplayType': '[nds character talent display type] list group',
  'talentsSignatureAbilitiesEnabled': False,
  'forcePowersDisplayType': '[nds character talent display type] list group',
  'dutyEnabled': False,
  'forceEnabled': False,
  'forceDiceEnabled': False,
  'moralityEnabled': False,
  'obligationEnabled': False,
  'superCharacteristicsEnabled': False,
  'heroicAbilitiesEnabled': False,
  'aemberEnabled': False,
  'favorsEna

In [15]:
from src.abilities_builder import ability_template_builder
from src.validator import validate_talent_or_ability
from src.utils import fill_template_with_openai
creature_description="A very angry bear-man that roams Desert Oases."
creature_name="Desert Bearman"
creature_type="Rival"
setting_description="A fantasy setting with primarily humans, and a tribe of them banished to the Desert."


ability_template_string = ability_template_builder(creature_name, creature_type, ["Melee", "Intimidation", "Knowledge (Forbidden)"], 7, 6, 5, setting_description, number_of_abilities=2)
print(ability_template_string)



Please generate 2 JSON-formatted NPC abilities (in JSON format, wrap all entries in a list please) for use in the Genesys RPG setting, increasing in their overall power as you go from weak to very strong (for the creature's CRs and type).
This is for a Combat CR 7, General CR 6, and Social CR 5 creature, of type Rival, called Desert Bearman, with skills Melee, Intimidation, Knowledge (Forbidden). The setting this creature is found in is as follows: A fantasy setting with primarily humans, and a tribe of them banished to the Desert.
Here are some sample abilities; their format should NOT be followed for your generation (see below for the format to follow).

{'name': 'Quiet Killer', 'description': 'May use Stealth instead of Melee or Brawl when making combat checks against unaware targets.'}
{'name': 'Knowledge is Dangerous', 'description': 'Adds :success::threat: to the results of Knowledge: Science checks other characters make in the encounter.'}
{'name': 'Los Scorpiones Boss', 'descri

In [4]:
print("Filling ability template with OpenAI, expect this to take about 15 seconds.")
generated_character_data = fill_template_with_openai(ability_template_string)

Filling ability template with OpenAI, expect this to take about 15 seconds.


In [8]:
print(generated_character_data)

[
    {
        "name": "Desert Adaptation",
        "tier": 2,
        "activation": false,
        "ranked": false,
        "description": "The Desert Bearman is adapted to the harsh desert environment. It is immune to the effects of extreme heat and can go for extended periods without water."
    },
    {
        "name": "Forbidden Knowledge",
        "tier": 4,
        "activation": true,
        "turn": "Action",
        "ranked": true,
        "description": "The Desert Bearman has access to ancient, forbidden knowledge. It can use this knowledge to gain an advantage in combat or social situations, adding a boost die to any related checks."
    }
]


In [18]:
print("Validating character data")
generated_ability_data_as_dict = validate_talent_or_ability(generated_character_data)

Validating character data
this passes all talent/ability validation steps
this passes all talent/ability validation steps


In [7]:
#sort by string representation


In [12]:
from src.utils import get_character
get_character("6490884518fddf53570b0916", bearer_token)

{'_id': '6490884518fddf53570b0916',
 'createdDate': '2023-06-19T16:54:29.993Z',
 'modifiedDate': '2023-06-19T16:54:29.993Z',
 'status': 1,
 'name': 'Feral Dragon',
 'lookupName': 'feral dragon',
 'image': 'https://unbound-legends.imgix.net/character2/genesys-T-jjr.jpg?auto=format',
 'notes': 'Imported from [DylanPetrusma.com](http://genesysadversaries.dylanpetrusma.com/)',
 'privateNotes': '',
 'rulesSystem': '[rules] nds',
 'characterType': '[character type] npc',
 'configuration': {'diceTheme': '[dice theme] genesys',
  'gameTheme': '[game theme] genesys terrinoth',
  'ndsCharacterType': '[nds character type] nemesis',
  'talentsDisplayType': '[nds character talent display type] list group',
  'talentsSignatureAbilitiesEnabled': False,
  'forcePowersDisplayType': '[nds character talent display type] list group',
  'dutyEnabled': False,
  'forceEnabled': False,
  'forceDiceEnabled': False,
  'moralityEnabled': False,
  'obligationEnabled': False,
  'superCharacteristicsEnabled': False